In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import mysql.connector as sql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import openpyxl

In [88]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
usd_eur = yf.download('USDEUR=X', start='2019-01-01', end='2024-01-01')

# minute data
#usd_eur = yf.download('USDEUR=X', period='5d', interval='5m')

In [137]:
usd_eur.index = pd.to_datetime(usd_eur.index)

In [ ]:
usd_eur

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(usd_eur.index.astype(str), usd_eur['Close'], label='Close Price', color='b')
plt.title('Close Price Over Time', fontsize=14)
plt.xlabel('Record Date', fontsize=12)
plt.ylabel('Close Price', fontsize=12)
plt.legend()
num_ticks = 10  
plt.gca().set_xticks(plt.gca().get_xticks()[::len(usd_eur)//num_ticks])
plt.xticks(rotation=45)
plt.show()

In [ ]:
usd_eur.isnull().sum()

In [132]:
train_size = int(len(usd_eur) * 0.8)
train = usd_eur[:train_size] 
test = usd_eur[train_size:] 

Perform Seasonal Decomposition to identify recurring patterns

In [ ]:
result = seasonal_decompose(usd_eur['Close'],model='multiplicative', period=24)
fig = plt.figure()
fig = result.plot()
fig.show()

In [ ]:
model = auto_arima(train['Close'], seasonal=True, m=60, suppress_warnings=True)
print(model.order)

In [ ]:
model = SARIMAX(train['Close'], order=(1, 1, 3), seasonal_order=(1, 1, 3, 60))
fitted = model.fit()
fitted.summary()   

In [ ]:


forecast = fitted.get_forecast(steps=len(test))

# Get predicted mean from the forecast
forecast_mean = forecast.predicted_mean

# Create a date range for the forecast, starting from the last date in the training set
forecast_index = pd.date_range(start=usd_eur.index[train_size - 1], periods=len(test) + 1, freq='T')[1:]

# Convert the forecasted values to a pandas Series and align with the forecast index
forecast_series = pd.Series(forecast_mean.values, index=forecast_index)

# Step 4: Plot the full data (actual and forecast)
plt.figure(figsize=(10, 6))

# Plot the actual data (all 1303 rows)
plt.plot(usd_eur['Close'], label='Actual Data', color='blue')

# Plot the forecasted data
plt.plot(forecast_series, label='Forecasted Data', color='orange', linestyle='--')

# Add titles and labels
plt.title('USD to EUR Exchange Rate Forecast')
plt.xlabel('Date')
plt.ylabel('Close Price')

# Add legend
plt.legend()

# Show the plot
plt.show()